# Taller 06: Bases de Datos Vectoriales  
**ICCD753 Recuperación de Información – Prof. Iván Carrera (2024-B, EPN-FIS)**  

**Fecha de Entrega:** Martes 14 de enero de 2025  
**Integrantes:**  
- Dilan Andrade  
- Hernán Sánchez  
- Galo Tarapués  

---

### **Objetivo del Taller**
Implementar y comparar estrategias de recuperación de información utilizando TF-IDF, BM25 y embeddings con bases de datos vectoriales, evaluando su relevancia y eficiencia en un dataset textual real.


# Parte1 Recuperación con TF-IDF 

### **Parte 1: Importación de Librerías para TF-IDF**
En esta celda se importan las librerías fundamentales para la implementación de la recuperación de información utilizando el modelo TF-IDF. Estas librerías permiten:

1. **`pandas`**: Manejar y procesar el dataset estructurado del taller.
2. **`TfidfVectorizer`** (de `scikit-learn`): Transformar los textos en una matriz TF-IDF, donde cada documento es representado como un vector en un espacio de palabras.
3. **`cosine_similarity`** (de `scikit-learn`): Calcular la similitud entre vectores de documentos y consultas, para determinar qué tan relevante es un documento en relación con la consulta.



In [4]:
# Importar librerías necesarias
import pandas as pd  # Para la manipulación de datos
from sklearn.feature_extraction.text import TfidfVectorizer  # Para calcular la matriz TF-IDF
from sklearn.metrics.pairwise import cosine_similarity  # Para calcular similitud entre vectores


### **Parte 2: Carga y Preprocesamiento del Dataset**
En esta celda se realiza la carga y preprocesamiento inicial del dataset `Wikipedia Movie Plots`. Los pasos incluyen:

1. **Definir la ruta del dataset**: Se especifica la ubicación del archivo CSV (`wiki_movie_plots_deduped.csv`).
2. **Cargar el dataset**: Se utiliza `pandas` para cargar el archivo CSV en un DataFrame, que permite manipular y analizar los datos.
3. **Eliminación de filas inválidas**: Se eliminan las filas que no contienen información válida en la columna `Plot`, ya que esta información es esencial para la tarea de recuperación.
4. **Verificación de datos**: Se muestran las primeras filas del DataFrame para asegurarse de que la carga y el preprocesamiento sean correctos.




In [6]:
# Definir la ruta del dataset
ruta_dataset = "wiki_movie_plots_deduped.csv"

# Cargar el archivo CSV en un DataFrame de pandas
df = pd.read_csv(ruta_dataset)

# Eliminar filas sin un 'Plot' válido
df = df.dropna(subset=['Plot'])

# Mostrar las primeras filas para verificar la carga de datos
df.head()


,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot
0,1901,Kansas Saloon Smashers,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Kansas_Saloon_Sm...,"A bartender is working at a saloon, serving dr..."
1,1901,Love by the Light of the Moon,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Love_by_the_Ligh...,"The moon, painted with a smiling face hangs ov..."
2,1901,The Martyred Presidents,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/The_Martyred_Pre...,"The film, just over a minute long, is composed..."
3,1901,"Terrible Teddy, the Grizzly King",American,Unknown,NaN,unknown,"https://en.wikipedia.org/wiki/Terrible_Teddy,_...",Lasting just 61 seconds and consisting of two ...
4,1902,Jack and the Beanstalk,American,"George S. Fleming, Edwin S. Porter",NaN,unknown,https://en.wikipedia.org/wiki/Jack_and_the_Bea...,The earliest known adaptation of the classic f...


### **Parte 3: Combinación de Título y Trama para el Modelo TF-IDF**
En esta celda se integra la información del título y la trama de cada película en una nueva columna llamada `texto_completo`. Este paso es crucial para que el modelo TF-IDF considere tanto el título como el contenido de la trama durante la extracción de características. 

#### Detalles clave:
1. **Integración de datos**: Se crea la columna `texto_completo` combinando el título (`Title`) y la trama (`Plot`) de cada película.
2. **Posibilidad de preprocesamiento adicional**:
   - Conversión a minúsculas para normalizar el texto.
   - Aplicación de expresiones regulares, lematización o stemming para mejorar la calidad del texto procesado.




In [7]:
# Combinar el título y la trama en una nueva columna 'texto_completo'
df['texto_completo'] = df['Title'].astype(str) + " " + df['Plot'].astype(str)



### **Parte 4: Configuración y Generación de la Matriz TF-IDF**
En esta celda se configura y genera la matriz TF-IDF para representar los documentos del dataset como vectores numéricos, listos para ser utilizados en cálculos de similitud.

#### Detalles clave:
1. **Configuración del `TfidfVectorizer`**:
   - `stop_words='english'`: Elimina palabras comunes en inglés (stopwords) que no aportan valor semántico.
   - `max_features=20000`: Limita el vocabulario a las 20,000 palabras más importantes según la frecuencia.
   - `sublinear_tf=True`: Aplica una escala logarítmica a las frecuencias para reducir el impacto de palabras extremadamente frecuentes.
   - `ngram_range=(1, 2)`: Captura unigramas (palabras individuales) y bigramas (pares de palabras consecutivas).
   
2. **Transformación de los documentos**:
   - Se ajusta el vectorizador a la columna `texto_completo`, generando una matriz TF-IDF.
   - Cada fila de la matriz representa un documento, y cada columna, una palabra o bigrama en el vocabulario.

3. **Salida**:
   - La forma de la matriz (`n_documentos`, `n_features`) es impresa para verificar la correcta generación.


In [8]:
# Configurar el vectorizador TF-IDF con parámetros ajustados
vectorizador_tfidf = TfidfVectorizer(
    stop_words='english',  # Remover stopwords en inglés
    max_features=20000,  # Limitar el vocabulario a 20,000 palabras
    sublinear_tf=True,  # Usar escala logarítmica para frecuencias
    ngram_range=(1, 2)  # Capturar unigramas y bigramas
)

# Generar la matriz TF-IDF ajustando el vectorizador a la columna 'texto_completo'
matriz_tfidf = vectorizador_tfidf.fit_transform(df['texto_completo'])

# Imprimir la forma de la matriz para verificar
print("Matriz TF-IDF generada con forma:", matriz_tfidf.shape)


Matriz TF-IDF generada con forma: (34886, 20000)


### **Parte 5: Función para Realizar Búsquedas con TF-IDF**
Esta celda define una función que permite realizar búsquedas en el dataset utilizando el modelo TF-IDF. Dada una consulta de texto, la función encuentra los documentos más relevantes mediante el cálculo de la similitud de coseno entre la consulta y los documentos del dataset.

#### Detalles clave:
1. **Transformación de la consulta**:
   - La consulta se transforma en un vector TF-IDF utilizando los mismos parámetros con los que se entrenó el vectorizador original.
   
2. **Cálculo de similitud**:
   - Se calcula la similitud de coseno entre el vector de la consulta y todos los documentos del dataset.
   
3. **Ordenamiento de resultados**:
   - Los documentos se ordenan en función de su puntuación de similitud en orden descendente.
   - Se retornan los índices de los documentos más relevantes y sus puntuaciones.

#### Parámetros:
- `consulta` (str): Texto de la consulta a buscar.
- `numero_resultados` (int): Número de documentos más relevantes a devolver (por defecto, 5).

#### Retorno:
- `indices_ordenados`: Índices de los documentos más relevantes.
- `puntuaciones`: Puntuaciones de similitud correspondientes a los documentos.


In [9]:
def buscar_con_tfidf(consulta, numero_resultados=5):
    # Convertir la consulta al vector TF-IDF
    vector_consulta = vectorizador_tfidf.transform([consulta])

    # Calcular similitud de coseno con todos los documentos
    similitudes = cosine_similarity(vector_consulta, matriz_tfidf).flatten()

    # Ordenar documentos por puntuación de similitud (descendente)
    indices_ordenados = similitudes.argsort()[::-1][:numero_resultados]
    puntuaciones = similitudes[indices_ordenados]

    return indices_ordenados, puntuaciones


### **Parte 6: Ejemplo de Consulta con TF-IDF**
En esta celda se realiza una consulta de ejemplo utilizando la función `buscar_con_tfidf`, previamente definida. El propósito es recuperar los documentos más relevantes para una consulta específica y mostrar información útil sobre ellos.

#### Detalles clave:
1. **Consulta de ejemplo**:
   - La consulta `"dinosaurs"` se utiliza para probar el sistema de búsqueda.
   
2. **Recuperación de documentos**:
   - La función `buscar_con_tfidf` retorna los índices y las puntuaciones de los documentos más relevantes en el dataset.
   
3. **Visualización de resultados**:
   - Por cada documento relevante, se imprime:
     - **Rango**: Posición del documento en el ranking.
     - **Título**: El título de la película.
     - **Similitud**: Puntuación de similitud calculada con el modelo TF-IDF.
     - **Trama**: Los primeros 200 caracteres de la trama, para evitar saturar la salida.



In [10]:
# Ejemplo de consulta
consulta_ejemplo = "dinosaurs"

# Obtener los documentos más similares
indices_encontrados, puntuaciones_encontradas = buscar_con_tfidf(consulta_ejemplo, numero_resultados=5)

# Mostrar los resultados
print(f"Resultados para la consulta: '{consulta_ejemplo}'\n")
for rank, (indice, puntuacion) in enumerate(zip(indices_encontrados, puntuaciones_encontradas), start=1):
    print(f"Rank {rank}")
    print(f"Título (Title): {df.loc[indice, 'Title']}")
    print(f"Similitud: {puntuacion:.4f}")
    # Mostrar los primeros 200 caracteres de la trama
    print(f"Trama (Plot) (primeros 200 caracteres): {df.loc[indice, 'Plot'][:200]}...")
    print("-" * 50)


Resultados para la consulta: 'dinosaurs'

Rank 1
Título (Title): Theodore Rex
Similitud: 0.3514
Trama (Plot) (primeros 200 caracteres): In an alternate futuristic society where humans and anthropomorphic dinosaurs co-exist, a tough police detective named Katie Coltraine (Whoopi Goldberg) is paired with a Tyrannosaurus named Theodore R...
--------------------------------------------------
Rank 2
Título (Title): Dinosaurs! – A Fun-Filled Trip Back in Time!
Similitud: 0.2623
Trama (Plot) (primeros 200 caracteres): The video—with beginning scenes filmed in 1987—begins with a young boy named Phillip (played by Fred Savage) sitting in his bedroom, listening to loud music, and struggling to find an idea for a class...
--------------------------------------------------
Rank 3
Título (Title): We're Back! A Dinosaur's Story
Similitud: 0.2593
Trama (Plot) (primeros 200 caracteres): In present-day New York City, an Eastern bluebird named Buster runs away from his siblings and he meets an intellige

## Reflexión Parte 1: Recuperación con TF-IDF
En esta parte, implementamos un modelo TF-IDF para calcular la relevancia de términos dentro del dataset de tramas de películas. El enfoque resultó efectivo para identificar documentos relacionados directamente con las palabras clave, aunque su capacidad es limitada frente a términos no exactos o con ambigüedades.


# Parte 2 Recuperación con BM25

### **Parte 1: Importación de Librerías para Configuración de Elasticsearch**
En esta celda se importan las librerías necesarias para trabajar con Elasticsearch, así como para facilitar el manejo y procesamiento de datos.

#### Detalles clave:
1. **`Elasticsearch`**:
   - Permite interactuar con un clúster de Elasticsearch para crear índices, almacenar documentos y realizar consultas.
   
2. **`pandas`**:
   - Facilita la manipulación del dataset para indexarlo en Elasticsearch.

3. **`tqdm`**:
   - Proporciona barras de progreso para monitorear la indexación de documentos.

4. **`time`**:
   - Se utiliza para medir tiempos de ejecución o manejar retrasos si es necesario.



In [11]:
# Importar librerías necesarias
from elasticsearch import Elasticsearch  # Para interactuar con Elasticsearch
import pandas as pd  # Para manejar y procesar datos
from tqdm import tqdm  # Para mostrar barras de progreso
import time  # Para medir tiempos de ejecución


### **Parte 2: Conexión con Elasticsearch**
En esta celda se configura la conexión al servidor local de Elasticsearch para asegurarse de que esté listo para indexar documentos y realizar consultas.

#### Detalles clave:
1. **Conexión**:
   - Se establece la conexión al servidor de Elasticsearch utilizando la URL `http://localhost:9200`.
   
2. **Verificación**:
   - Se utiliza el método `ping()` para comprobar si el servidor de Elasticsearch está accesible.
   - Se imprime un mensaje indicando si la conexión fue exitosa o fallida.


In [12]:
# Conectar con Elasticsearch

# Configuramos la conexión con Elasticsearch
es = Elasticsearch(['http://localhost:9200'])

# Verificamos la conexión
if es.ping():
    print("Conectado a Elasticsearch")
else:
    print("No se pudo conectar a Elasticsearch")

Conectado a Elasticsearch


### **Parte 3: Configuración del Índice en Elasticsearch**
En esta celda se configura y crea un índice en Elasticsearch para almacenar los documentos del dataset, con ajustes específicos para mejorar la búsqueda basada en BM25.

#### Detalles clave:
1. **Nombre del índice**:
   - Se define el nombre del índice como `"movie_plots"`.

2. **Configuración del índice**:
   - **`settings`**:
     - `number_of_shards`: Número de fragmentos del índice (1 en este caso).
     - `number_of_replicas`: Número de réplicas del índice (0 para evitar réplicas en este ejemplo).
     - `analysis`: Define un analizador personalizado con soporte para stopwords en inglés.
   - **`mappings`**:
     - Especifica los campos del índice (`Title` y `Plot`) como texto (`text`) y les asigna el analizador personalizado.

3. **Creación del índice**:
   - Verifica si el índice ya existe:
     - Si no existe, lo crea utilizando la configuración definida.
     - Si ya existe, imprime un mensaje indicando su existencia.



In [13]:
# Configuración del índice para almacenar los documentos
index_name = "movie_plots"

# Definir el esquema del índice con mapeos para BM25
index_config = {
    "settings": {
        "number_of_shards": 1,  # Número de fragmentos
        "number_of_replicas": 0,  # Número de réplicas
        "analysis": {
            "analyzer": {
                "custom_analyzer": {
                    "type": "standard",  # Tipo de analizador
                    "stopwords": "_english_"  # Stopwords en inglés
                }
            }
        }
    },
    "mappings": {
        "properties": {
            "Title": {"type": "text", "analyzer": "custom_analyzer"},  # Campo 'Title'
            "Plot": {"type": "text", "analyzer": "custom_analyzer"}  # Campo 'Plot'
        }
    }
}

# Crear el índice en Elasticsearch
if not es.indices.exists(index=index_name):
    es.indices.create(index=index_name, body=index_config)
    print(f"Índice '{index_name}' creado exitosamente.")
else:
    print(f"El índice '{index_name}' ya existe.")


El índice 'movie_plots' ya existe.


### **Parte 4: Carga del Dataset e Indexación de Documentos**
En esta celda se carga el dataset, se filtran las filas necesarias, y los documentos se indexan en el índice previamente configurado en Elasticsearch.

#### Detalles clave:
1. **Carga del dataset**:
   - Se lee el archivo `wiki_movie_plots_deduped.csv` en un DataFrame de `pandas`.
   
2. **Filtrado**:
   - Se eliminan las filas que tienen valores nulos en la columna `Plot`, ya que esta columna es esencial para las búsquedas.



In [14]:
# Cargar el dataset
ruta_dataset = "wiki_movie_plots_deduped.csv"
df = pd.read_csv(ruta_dataset)

# Filtrar filas con valores nulos en 'Plot'
df = df.dropna(subset=['Plot'])

# Insertar documentos en el índice de Elasticsearch
for i, row in tqdm(df.iterrows(), total=len(df), desc="Indexando documentos en Elasticsearch"):
    doc = {
        "Title": row['Title'],  # Título de la película
        "Plot": row['Plot']  # Trama de la película
    }
    es.index(index=index_name, id=i, body=doc)

# Confirmar la cantidad de documentos indexados
print(f"Se han indexado {len(df)} documentos en el índice '{index_name}'.")


Indexando documentos en Elasticsearch: 100%|██████████| 34886/34886 [38:04<00:00, 15.27it/s]  

Se han indexado 34886 documentos en el índice 'movie_plots'.


### **Parte 5: Función para Realizar Búsquedas con BM25**
Esta celda define una función para realizar búsquedas en el índice de Elasticsearch utilizando el modelo de recuperación BM25. La función permite encontrar los documentos más relevantes según los términos de búsqueda proporcionados.

#### Detalles clave:
1. **Consulta al índice**:
   - Se utiliza una consulta `match` en el campo `Plot` para buscar coincidencias basadas en los términos de la consulta.

2. **Parámetros de la función**:
   - `consulta` (str): Términos de búsqueda que se usarán para recuperar documentos.
   - `numero_resultados` (int): Cantidad máxima de resultados relevantes a devolver (por defecto, 5).

3. **Procesamiento de resultados**:
   - La respuesta de Elasticsearch incluye los documentos más relevantes y sus puntuaciones de relevancia (`_score`).
   - La función organiza los resultados en una lista de diccionarios con las claves `Title`, `Plot` y `Score`.

4. **Retorno**:
   - Una lista de documentos relevantes, cada uno con su título, trama y puntuación de relevancia.


In [15]:
def buscar_con_bm25(consulta, numero_resultados=5):
    """
    Realiza una consulta al índice en Elasticsearch utilizando BM25.
    
    Args:
        consulta (str): Términos de búsqueda.
        numero_resultados (int): Número de resultados a retornar.
    
    Returns:
        resultados (list): Lista de documentos relevantes con título, trama y puntuación.
    """
    # Definir la consulta para Elasticsearch
    query = {
        "query": {
            "match": {
                "Plot": consulta  # Buscar coincidencias en la trama
            }
        },
        "size": numero_resultados  # Número de resultados a retornar
    }

    # Realizar la consulta al índice
    respuesta = es.search(index=index_name, body=query)

    # Procesar los resultados
    resultados = []
    for hit in respuesta['hits']['hits']:
        resultados.append({
            "Title": hit['_source']['Title'],  # Título del documento
            "Plot": hit['_source']['Plot'],  # Trama del documento
            "Score": hit['_score']  # Puntuación de relevancia
        })
    
    return resultados


### **Parte 6: Ejemplo de Consulta con BM25**
En esta celda se realiza un ejemplo de consulta al índice en Elasticsearch utilizando el modelo BM25. El objetivo es recuperar los documentos más relevantes para una consulta específica y mostrar detalles útiles sobre ellos.

#### Detalles clave:
1. **Consulta de ejemplo**:
   - La consulta `"dinosaurs"` se utiliza para probar la funcionalidad del sistema de búsqueda.

2. **Recuperación de documentos**:
   - La función `buscar_con_bm25` devuelve los documentos más relevantes y sus puntuaciones.

3. **Visualización de resultados**:
   - Por cada documento relevante, se imprime:
     - **Rango**: La posición del documento en el ranking.
     - **Título**: El título de la película.
     - **Puntuación (Score)**: La relevancia del documento calculada por BM25.
     - **Trama**: Los primeros 200 caracteres de la trama, para evitar saturar la salida.



In [16]:
# Ejemplo de consulta con BM25
consulta_ejemplo = "dinosaurs"

# Obtener los documentos más relevantes
resultados_bm25 = buscar_con_bm25(consulta_ejemplo, numero_resultados=5)

# Mostrar los resultados
print(f"Resultados para la consulta: '{consulta_ejemplo}'\n")
for rank, resultado in enumerate(resultados_bm25, start=1):
    print(f"Rank {rank}")
    print(f"Título: {resultado['Title']}")
    print(f"Score: {resultado['Score']:.4f}")
    print(f"Trama (primeros 200 caracteres): {resultado['Plot'][:200]}...")
    print("-" * 50)


Resultados para la consulta: 'dinosaurs'

Rank 1
Título: We're Back! A Dinosaur's Story
Score: 12.8495
Trama (primeros 200 caracteres): In present-day New York City, an Eastern bluebird named Buster runs away from his siblings and he meets an intelligent orange Tyrannosaurus named Rex, who is playing golf. He explains to Buster that h...
--------------------------------------------------
Rank 2
Título: Theodore Rex
Score: 11.7067
Trama (primeros 200 caracteres): In an alternate futuristic society where humans and anthropomorphic dinosaurs co-exist, a tough police detective named Katie Coltraine (Whoopi Goldberg) is paired with a Tyrannosaurus named Theodore R...
--------------------------------------------------
Rank 3
Título: Future War
Score: 11.5786
Trama (primeros 200 caracteres): Future War begins aboard a spaceship undergoing a revolt. A man enters and activates an escape pod which travels to Earth and crashes into the Pacific Ocean. The pod contains “The Runaway”, a human sl...


## Reflexión Parte 2: Recuperación con BM25
Usamos Elasticsearch para implementar BM25, lo que permitió una recuperación más robusta que TF-IDF al considerar la saturación de términos y la longitud de los documentos. Esta técnica mostró mejores resultados al priorizar documentos más relevantes para consultas específicas como "dinosaurs".


# Parte 3: Recuperación con FAISS

#### **Parte 1: Preparación de datos**  
Esta etapa consiste en preparar los datos para su posterior procesamiento:  
- Se carga un dataset que contiene tramas y títulos de películas.  
- Se eliminan filas con valores nulos en la columna `Plot` para evitar errores.  
- Se crea una nueva columna, `texto_completo`, combinando el título y la trama de cada película. Esto asegura que toda la información relevante esté en un solo campo para la recuperación posterior.  

In [ ]:
# Parte 1: Preparación de datos
import pandas as pd
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
from tqdm import tqdm

# Cargar el dataset
ruta_dataset = "wiki_movie_plots_deduped.csv"
df = pd.read_csv(ruta_dataset)

# Eliminar filas con valores nulos en la columna 'Plot'
df = df.dropna(subset=['Plot'])

# Crear la columna 'texto_completo' combinando 'Title' y 'Plot'
df['texto_completo'] = df['Title'].astype(str) + " " + df['Plot'].astype(str)

# Verificar que 'texto_completo' fue creado correctamente
print("Columnas disponibles:", df.columns)
print("Ejemplo de texto completo:", df['texto_completo'].head())

# Parte 2: Generación de embeddings con SentenceTransformer
modelo_embeddings = SentenceTransformer('all-MiniLM-L6-v2')  # Puedes usar otro modelo según necesidad

# Convertir los textos en vectores
textos = df['texto_completo'].tolist()

#### **Parte 2: Generación de embeddings con SentenceTransformer**  
En este paso, los datos procesados son convertidos a representaciones vectoriales llamadas embeddings:  
- Se utiliza el modelo `SentenceTransformer` para representar semánticamente los textos combinados.  
- El progreso de la generación de embeddings se muestra mediante una barra de carga (`tqdm`) para facilitar el monitoreo.  
- Estos embeddings serán usados más adelante para realizar búsquedas eficientes basadas en similitud.  


In [ ]:


# Generar embeddings y mostrar progreso
embeddings = []
for texto in tqdm(textos, desc="Generando embeddings"):
    embeddings.append(modelo_embeddings.encode(texto, convert_to_numpy=True))
embeddings = np.array(embeddings)

Columnas disponibles: Index(['Release Year', 'Title', 'Origin/Ethnicity', 'Director', 'Cast',
       'Genre', 'Wiki Page', 'Plot', 'texto_completo'],
      dtype='object')
Ejemplo de texto completo: 0    Kansas Saloon Smashers A bartender is working ...
1    Love by the Light of the Moon The moon, painte...
2    The Martyred Presidents The film, just over a ...
3    Terrible Teddy, the Grizzly King Lasting just ...
4    Jack and the Beanstalk The earliest known adap...
Name: texto_completo, dtype: object


Generando embeddings: 100%|██████████| 34886/34886 [36:17<00:00, 16.02it/s]  



Agregando embeddings al índice FAISS:


Indexando en FAISS: 100%|██████████| 34886/34886 [00:00<00:00, 135553.57it/s]



Índice FAISS creado con 34886 vectores.



#### **Parte 3: Configurar FAISS**  
Aquí se configura un índice FAISS para permitir búsquedas rápidas de similitud entre vectores:  
- Se utiliza la métrica L2 (distancia euclidiana cuadrada) para medir similitudes entre vectores.  
- Los embeddings generados previamente se añaden al índice en un proceso que muestra su progreso con una barra de carga (`tqdm`).  
- Una vez creado, el índice contiene todos los vectores necesarios para las búsquedas futuras.  


In [ ]:

# Parte 3: Configurar FAISS
# Crear el índice en FAISS (usamos L2 para búsqueda más rápida)
dimension = embeddings.shape[1]  # Dimensión de los vectores
indice_faiss = faiss.IndexFlatL2(dimension)

# Agregar los embeddings al índice con barra de progreso
print("\nAgregando embeddings al índice FAISS:")
for i in tqdm(range(len(embeddings)), desc="Indexando en FAISS"):
    indice_faiss.add(np.expand_dims(embeddings[i], axis=0))

print(f"\nÍndice FAISS creado con {indice_faiss.ntotal} vectores.")


#### **Parte 4: Realizar consultas en FAISS**  
En esta etapa, se ejecutan búsquedas en el índice FAISS:  
- Una consulta textual es convertida en un vector de embedding utilizando el mismo modelo de SentenceTransformer.  
- FAISS busca en su índice los vectores más cercanos al vector de la consulta y devuelve los resultados más relevantes.  
- Cada resultado incluye el título, la trama y una métrica de similitud (distancia).  

In [4]:
# Parte 4: Consultas en FAISS
def buscar_con_faiss(consulta, numero_resultados=5):
    """
    Realiza una consulta en FAISS buscando los vectores más cercanos.
    """
    # Convertir la consulta a un vector de embedding
    vector_consulta = modelo_embeddings.encode([consulta], convert_to_numpy=True)
    
    # Buscar los vecinos más cercanos en FAISS
    distancias, indices = indice_faiss.search(vector_consulta, numero_resultados)
    
    resultados = []
    for indice, distancia in zip(indices[0], distancias[0]):
        resultados.append({
            "Title": df.iloc[indice]['Title'],
            "Plot": df.iloc[indice]['Plot'],
            "Distancia": distancia
        })
    return resultados



#### **Parte 5: Comparar resultados de TF-IDF, BM25 y FAISS**  
En esta etapa, se comparan los resultados obtenidos utilizando las técnicas TF-IDF, BM25 y FAISS:  
- Para TF-IDF y BM25, se calculan las puntuaciones de relevancia de los documentos recuperados.  
- Para FAISS, se utiliza la métrica de distancia para identificar los documentos más cercanos.  
- Los cuatro primeros resultados de cada técnica son mostrados junto con sus métricas respectivas, permitiendo una comparación clara de su efectividad.  


In [5]:
# Ejemplo de consulta
consulta_ejemplo = "A park with cloned dinosaurs"
resultados_faiss = buscar_con_faiss(consulta_ejemplo, numero_resultados=5)

# Mostrar los resultados
print(f"\nResultados para la consulta: '{consulta_ejemplo}'\n")
for rank, resultado in enumerate(resultados_faiss, start=1):
    print(f"Rank {rank}")
    print(f"Título: {resultado['Title']}")
    print(f"Distancia: {resultado['Distancia']:.4f}")
    print(f"Trama (primeros 200 caracteres): {resultado['Plot'][:200]}...")
    print("-" * 50)


Resultados para la consulta: 'A park with cloned dinosaurs'

Rank 1
Título: Jurassic Park
Distancia: 0.6588
Trama (primeros 200 caracteres): Industrialist John Hammond and his bioengineering company, InGen, have created a theme park called Jurassic Park on Isla Nublar, a Costa Rican island, populated with cloned dinosaurs. After one of the...
--------------------------------------------------
Rank 2
Título: Dinosaurs! – A Fun-Filled Trip Back in Time!
Distancia: 1.0297
Trama (primeros 200 caracteres): The video—with beginning scenes filmed in 1987—begins with a young boy named Phillip (played by Fred Savage) sitting in his bedroom, listening to loud music, and struggling to find an idea for a class...
--------------------------------------------------
Rank 3
Título: Jurassic World
Distancia: 1.0311
Trama (primeros 200 caracteres): Brothers Zach and Gray Mitchell visit Isla Nublar, the site of the original Jurassic Park, where a new theme park named Jurassic World has operated for year


#### **Ejemplo de Consulta y Comparación**  
Se ejecuta una consulta específica, como "A park with cloned dinosaurs", y se compara el rendimiento de las tres técnicas.  
- Se presentan los títulos, tramas (truncadas para legibilidad) y métricas de relevancia para los cuatro mejores resultados obtenidos por cada técnica.  
- Esto permite evaluar cuál de las estrategias ofrece una recuperación más relevante para el usuario.

In [18]:
# Función para comparar resultados de TF-IDF, BM25 y FAISS
def comparar_resultados(consulta, numero_resultados=4):
    """
    Compara los resultados obtenidos con TF-IDF, BM25 y FAISS y muestra los 4 primeros de cada técnica.
    """
    print(f"\nComparación de resultados para la consulta: '{consulta}'\n")
    
    # TF-IDF
    indices_tfidf, puntuaciones_tfidf = buscar_con_tfidf(consulta, numero_resultados)
    resultados_tfidf = [
        {"Rank": rank + 1, 
         "Title": df.loc[indice, 'Title'], 
         "Score": puntuacion, 
         "Plot": df.loc[indice, 'Plot'][:200]}
        for rank, (indice, puntuacion) in enumerate(zip(indices_tfidf, puntuaciones_tfidf))
    ]
    print("Resultados con TF-IDF:")
    for resultado in resultados_tfidf:
        print(f"Rank {resultado['Rank']}: {resultado['Title']} (Score: {resultado['Score']:.4f})")
        print(f"Trama: {resultado['Plot']}...")
        print("-" * 50)
    
    # BM25
    resultados_bm25 = buscar_con_bm25(consulta, numero_resultados)
    print("\nResultados con BM25:")
    for rank, resultado in enumerate(resultados_bm25, start=1):
        print(f"Rank {rank}: {resultado['Title']} (Score: {resultado['Score']:.4f})")
        print(f"Trama: {resultado['Plot'][:200]}...")
        print("-" * 50)
    
    # FAISS
    resultados_faiss = buscar_con_faiss(consulta, numero_resultados)
    print("\nResultados con FAISS:")
    for rank, resultado in enumerate(resultados_faiss, start=1):
        print(f"Rank {rank}: {resultado['Title']} (Distancia: {resultado['Distancia']:.4f})")
        print(f"Trama: {resultado['Plot'][:200]}...")
        print("-" * 50)

# Ejemplo de comparación mostrando los 4 primeros resultados
consulta_ejemplo = "A park with cloned dinosaurs"
comparar_resultados(consulta_ejemplo, numero_resultados=4)



Comparación de resultados para la consulta: 'A park with cloned dinosaurs'

Resultados con TF-IDF:
Rank 1: Theodore Rex (Score: 0.2996)
Trama: In an alternate futuristic society where humans and anthropomorphic dinosaurs co-exist, a tough police detective named Katie Coltraine (Whoopi Goldberg) is paired with a Tyrannosaurus named Theodore R...
--------------------------------------------------
Rank 2: We're Back! A Dinosaur's Story (Score: 0.2416)
Trama: In present-day New York City, an Eastern bluebird named Buster runs away from his siblings and he meets an intelligent orange Tyrannosaurus named Rex, who is playing golf. He explains to Buster that h...
--------------------------------------------------
Rank 3: Jurassic Park (Score: 0.2376)
Trama: Industrialist John Hammond and his bioengineering company, InGen, have created a theme park called Jurassic Park on Isla Nublar, a Costa Rican island, populated with cloned dinosaurs. After one of the...
-----------------------------------